In [4]:
import streamlit as st
import pandas as pd
import pickle
import calendar
from datetime import datetime
import matplotlib.pyplot as plt

# Load the model and department label encoder
model = pickle.load(open('trained_model_month_only.pkl', 'rb'))
le_dept = pickle.load(open('label_encoder_dept.pkl', 'rb'))

departments = le_dept.classes_

# =======================
# Streamlit UI
# =======================
st.set_page_config(page_title="Department Performance Comparator", layout="centered")
st.title("📊 Department-wise Performance Comparison (Month-based Prediction)")

# Input: Month
month_input = st.text_input("Enter Month (e.g., Jan or March):")

# Predict button
if st.button("Predict Performance"):

    # Validate month input
    try:
        month_number = datetime.strptime(month_input.strip(), "%b").month
    except ValueError:
        try:
            month_number = datetime.strptime(month_input.strip(), "%B").month
        except ValueError:
            st.error("❌ Invalid month. Please enter full or short month name (e.g., Jan, February).")
            st.stop()

    # Create a DataFrame for all departments with the given month
    data = pd.DataFrame({
        'department': departments,
        'month': [month_number] * len(departments)
    })

    # Predict performance using only the month
    X = data[['month']]
    predictions = model.predict(X)

    # Convert predictions to numeric (assuming 'Good' = 1, 'Bad' = 0)
    performance_mapping = {'Good': 1, 'Bad': 0}
    data['performance_score'] = pd.to_numeric([performance_mapping.get(p, 0) for p in predictions])

    # Normalize to percentage
    min_score = data['performance_score'].min()
    max_score = data['performance_score'].max()
    if max_score - min_score == 0:
        data['performance_percent'] = 50
    else:
        data['performance_percent'] = 100 * (data['performance_score'] - min_score) / (max_score - min_score)

    data['performance_percent'] = data['performance_percent'].round(2)

    # =======================
    # Visualization
    # =======================
    st.subheader(f"📅 Predicted Performance - {calendar.month_name[month_number]}")

    fig, ax = plt.subplots(figsize=(10, 5))

    if data['performance_percent'].nunique() == 1:
        colors = ['gray'] * len(data)
    else:
        colors = plt.cm.viridis(data['performance_percent'] / 100)

    bars = ax.bar(data['department'], data['performance_percent'], color=colors)
    ax.set_ylabel("Predicted Performance (%)")
    ax.set_title("Department-wise Predicted Performance (Normalized)")
    ax.set_xticks(range(len(data['department'])))
    ax.set_xticklabels(data['department'], rotation=45, ha='right')
    ax.set_ylim(0, 110)

    for bar, score in zip(bars, data['performance_percent']):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 2, f"{score:.1f}%", ha='center', fontsize=9)

    st.pyplot(fig)

    with st.expander("📋 Show Performance Table"):
        st.dataframe(data[['department', 'performance_score', 'performance_percent']])

    # Debug/Insight
    st.write("🧮 Raw Predictions:")
    st.dataframe(pd.DataFrame({'Department': data['department'], 'Prediction': predictions}))


2025-04-14 17:42:58.495 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 17:42:58.497 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 17:43:00.489 
  command:

    streamlit run C:\Users\SAMYAK KHANDERAO\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-04-14 17:43:00.491 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 17:43:00.493 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 17:43:00.494 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-14 17:43:00.496 Thread 'MainThread': missing ScriptRunConte